In [1]:
from hojas import *
from scipy.stats import norm

In [2]:
# --------------------- APRENDIZAJE BAYESIANO CON DATA HOJAS / GAUSSIANA MULTIVARIABLE ---------------------
def mu_iter(n, muestras, mu_0, sigma_0, sigma):
    mu_n = (n*(sigma_0)*(np.sum(muestras)/n)) + (sigma*mu_0)
    mu_n /= ((n*(sigma_0))+sigma)
    return mu_n

def sigma_iter(n, sigma_0, sigma):
    sigma_n = sigma_0*sigma
    sigma_n /= ((n*(sigma_0))+sigma)
    return sigma_n


def iter_bayes(muestras, mu_0, sigma_0, sigma):
    mu_todos = []
    sigma_todos = []
    for i in range(1, len(muestras)+1):
        mu_n = mu_iter(i, muestras[:i], mu_0, sigma_0, sigma)
        mu_todos.append(mu_n)
        sigma_n = sigma_iter(i, sigma_0, sigma)
        sigma_todos.append(sigma_n)
    return mu_n, sigma_n


def buscar_mu_con_bayes(muestra, sep=0.7):
    muestra = np.random.permutation(muestra) 
    cant_agarro = int(len(muestra)*sep) # agarro un 70% de las muestras para entrenar y el resto para testear
    muestra_est = muestra[:cant_agarro]
    muestra_test = muestra[cant_agarro:]
    mu_0 = muestra_est[0]
    sigma_0 = 10
    sigma = np.var(muestra_est)

    mu, sigma = iter_bayes(muestra_est, mu_0, sigma_0, sigma)
    
    return mu, sigma, muestra_test


In [3]:
# --------------------- TEST APRENDIZAJE BAYESIANO CON DATA HOJAS ---------------------
def test_mu(m, mu1, des1, mu2, des2): #True:
    """
    True: pertenece a clase 1
    False: pertenece a clase 2
    """
    proba1 = norm.pdf(m, mu1, des1)
    proba2 = norm.pdf(m, mu2, des2)
    if proba1 >= proba2:
        return True
    return False

# BOOTSTRAPPING
def test_muestras(muestra1, muestra2):
    mu1, sigma1, muestra_test1 = buscar_mu_con_bayes(muestra1)
    mu2, sigma2, muestra_test2 = buscar_mu_con_bayes(muestra2)

    des1 = np.sqrt(sigma1)
    des2 = np.sqrt(sigma2)

    #buscar a cuantos separa mal la estimación conseguida
    m1_mal = 0
    for m in muestra_test1:
        if not test_mu(m, mu1, des1, mu2, des2):
            m1_mal += 1

    m2_mal = 0
    for m in muestra_test2:
        if test_mu(m, mu1, des1, mu2, des2):
            m2_mal += 1

    return m1_mal/len(muestra_test1),  m2_mal/len(muestra_test2) #erró sobre total


In [4]:
# ------ TEST MUESTRAS BIEN CLASIFICADAS LARGOS ------
error1, error2 = test_muestras(c1['largo'], c2['largo'])
print('------------ CLASE 1 ------------')
print(f'Acertó de largo clase 1 = {round(1-error1, 3)}') #acertó sobre total
print(f'Acertó de largo clase 2 = {round(1-error2, 3)}')

# ------ VER TOTAL DE MUESTRAS BIEN LARGOS ------
p_largo1 = 0.59 #proba de que la muestra sea de clase 1, son 332 de 554
p_largo2 = 0.41 #proba de que la muestra sea de clase 2, son 222 de 554
performance_largo = p_largo1*(1-error1) + p_largo2*(1-error2)
print(f'Performance de largos total = {round(performance_largo, 3)}')


# ------ TEST MUESTRAS BIEN CLASIFICADAS ANCHOS ------
print('\n------------ CLASE 2 ------------')
error1, error2 = test_muestras(c1['ancho'], c2['ancho'])
print(f'Acertó de ancho clase 1 = {round(1-error1, 3)}')
print(f'Acertó de ancho clase 2 = {round(1-error2, 3)}')

# ------ VER TOTAL DE MUESTRAS BIEN ANCHOS ------
p_ancho1 = 0.59
p_ancho2 = 0.41
performance_largo = p_ancho1*(1-error1) + p_ancho2*(1-error2)
print(f'Performance de anchos total = {round(performance_largo, 3)}')



------------ CLASE 1 ------------
Acertó de largo clase 1 = 0.74
Acertó de largo clase 2 = 0.791
Performance de largos total = 0.761

------------ CLASE 2 ------------
Acertó de ancho clase 1 = 0.67
Acertó de ancho clase 2 = 0.836
Performance de anchos total = 0.738


In [6]:
# ------ TEST PERFORMANCE ------
def perfo(p_c1, p_c2, err1, err2):
    perform = p_c1*(1-err1) + p_c2*(1-err2)
    return perform 

def test_performance(muestra1, muestra2, p1, p2, name):
    performs = []
    perfo1 = []
    perfo2 = []
    for _ in range(100):
        err1, err2 = test_muestras(muestra1, muestra2)
        performs.append(perfo(p1, p2, err1, err2))
        perfo1.append(1-err1)
        perfo2.append(1-err2)

    print(f'Performance {name}1 = {np.mean(perfo1)} +- {np.sqrt(np.var(perfo1))/(np.sqrt(10))}') # /sqrt(10) para que sea error estandar
    print(f'Performance {name}2 = {np.mean(perfo2)} +- {np.sqrt(np.var(perfo2))/(np.sqrt(10))}')

    print(f'Promedio perfo = {np.mean(performs)}')
    print(f'Desv estandar perfo = {np.std(performs)}')
    print(f'Performance {name} = {np.mean(performs)} +- {np.sqrt(np.var(performs))}')

print('\n ------------ TEST PERFORMANCE LARGOS ------------')
test_performance(c1['largo'], c2['largo'], p_largo1, p_largo2, 'largo')

print('\n ------------ TEST PERFORMANCE ANCHOS ------------')
test_performance(c1['ancho'], c2['ancho'], p_ancho1, p_ancho2, 'ancho')



 ------------ TEST PERFORMANCE LARGOS ------------


Performance largo1 = 0.7563 +- 0.012834757496735188
Performance largo2 = 0.7944776119402985 +- 0.014560541068382534
Promedio perfo = 0.7719528208955224
Desv estandar perfo = 0.026358744477323028
Performance largo = 0.7719528208955224 +- 0.026358744477323028

 ------------ TEST PERFORMANCE ANCHOS ------------
Performance ancho1 = 0.6577999999999998 +- 0.012857526978388958
Performance ancho2 = 0.7658208955223881 +- 0.015098916444240417
Promedio perfo = 0.7020885671641791
Desv estandar perfo = 0.030431411304688086
Performance ancho = 0.7020885671641791 +- 0.030431411304688086


In [ ]:
# ------ TEST PERFORMANCE DE OTRA MANERA, POR LOTES ------
def test2_perfo(muestra1, muestra2, p1, p2, name):
    divido_en = 22
    perfors = []
    perfo1 = []
    perfo2 = []
    for i in range(10):
        err1, err2 = test_muestras(muestra1[i*divido_en:(i+1)*divido_en], muestra2[i*divido_en:(i+1)*divido_en])
        perfors.append(perfo(p1, p2, err1, err2))
        perfo1.append(1-err1)
        perfo2.append(1-err2)

    print(f'Performance {name}1 = {np.mean(perfo1)} +- {np.sqrt(np.var(perfo1))/(np.sqrt(10))}')
    print(f'Performance {name}2 = {np.mean(perfo2)} +- {np.sqrt(np.var(perfo2))/(np.sqrt(10))}')
    print(f'Performance {name} = {np.mean(perfors)} +- {np.sqrt(np.var(perfors))/(np.sqrt(10))}')#/raiz(10) para tener el error estandar, 
    #funciona solo porque los lotes son indeo entre sí, si son dependientes (por ej se repiten entre lotes) el error est tiene mult la raiz, no dividida
    #EJ = si agarramos una muestra para test y el resto para estimar, y así con cada muestra, el error despues sería con la raiz(N-1) multiplicada

'''
OBS = la media (osea la performance) nos dice que tan superpuestas están las campanas, porque si tenes una media de 0,5 por ejemplo, 
implica que la mitad de las vece encuentra bien su distribución y la otra mitad no
Además, el desv estandar te dice si esa media es significativa, es decir, si te da media 0,7 y desv est bajo por
lo que la gaussiana que la describe no supera el 0,5 --> tu media es así porque tu datos están cercanos a ese valor, no porque
algunos dieron muy alto y otros muy bajo. En cambio, si este es muy alto varía mucho.
'''

print('\n ------------ TEST2 PERFORMANCE LARGOS ------------')
test2_perfo(c1['largo'][:len(c2['largo'])], c2['largo'], p_largo1, p_largo2, 'largo')

print('\n ------------ TEST2 PERFORMANCE ANCHOS ------------')
test2_perfo(c1['ancho'][:len(c2['largo'])], c2['ancho'], p_ancho1, p_ancho2, 'ancho')
    


 ------------ TEST2 PERFORMANCE LARGOS ------------


Performance largo1 = 0.7857142857142858 +- 0.058028845747399714
Performance largo2 = 0.7285714285714286 +- 0.05872801368884134
Performance largo = 0.7622857142857142 +- 0.04244189305054954

 ------------ TEST2 PERFORMANCE ANCHOS ------------
Performance ancho1 = 0.5857142857142856 +- 0.08439460278709675
Performance ancho2 = 0.8 +- 0.05421047417431507
Performance ancho = 0.6735714285714285 +- 0.05752417415526188


In [ ]:
# # ------ GRAFICAR CURVAS DE NIVEL ------
# # Clase 1
# mean = np.array([esp_largo_c1, esp_ancho_c1])
# cov = np.array([[var_largo_c1, cov_clase1], [cov_clase1, var_ancho_c1]])  # Matriz de covarianza
# # Clase 2
# mean = np.array([esp_largo_c2, esp_ancho_c2])
# cov = np.array([[var_largo_c2, cov_clase2], [cov_clase2, var_ancho_c2]])  # Matriz de covarianza

# # Generar datos para el gráfico de curvas de nivel
# x, y = np.mgrid[np.min(c1['largo']):np.max(c1['largo']):.01, np.min(c1['ancho']):np.max(c1['ancho']):.01]
# pos = np.dstack((x, y))
# z = np.random.multivariate_normal(mean, cov, 5000).T

# # Graficar las curvas de nivel
# plt.figure(figsize=(8, 6))
# plt.contourf(x, y, np.exp(-(z @ np.linalg.inv(cov) @ z.T).diagonal()/2), cmap='viridis')
# plt.colorbar()
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.title('Curvas de nivel de una distribución gaussiana de dos variables')
# plt.show()